In [ ]:
!pip install transformers pandas nltk emoji torch --upgrade


In [ ]:
from transformers import pipeline
import pandas as pd

In [8]:
type_classifier = pipeline(    "zero-shot-classification",    model="MoritzLaurer/DeBERTa-v3-base-mnli-fever-anli",    multi_label=False)
type_labels =  [ "technology", "News" , "Job search", "Editorial", "Blog", "Webinar", "Bank Related"]

urgency_classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")
urgency_labels = ["blocker", "major", "normal", "minor"]

sentiment_classifier = pipeline("sentiment-analysis", model="cardiffnlp/twitter-roberta-base-sentiment-latest")

Device set to use cpu
Device set to use cpu
Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cpu


In [11]:
file_path = 'emails.pkl'

df = pd.read_pickle(file_path)
df.head(10)

,uid,to,from,subject,content,timestamp
0,16458,yashjoshi30052002@gmail.com,Google <no-reply@accounts.google.com>,security alert,image google fetchemail granted access g...,2025-04-10 10:21:16 UTC
1,16459,yashjoshi30052002@gmail.com,redBus <greetings@travel.e-redbus.in>,yash plan festive journey :blossom:,redbus welcome plan bus journey new year...,2025-04-10 16:24:19 UTC+05:30
2,16460,yashjoshi30052002@gmail.com,Mail Delivery Subsystem <mailer-daemon@googlem...,delivery status notification failure,address found message wasnt delivered re...,2025-04-10 03:55:55 UTC-07:00
3,16461,yashjoshi30052002@gmail.com,Semrush Team <news@team.semrush.com>,quick onpage seo fixes today,higher rankings ever heard onpage seo it...,2025-04-10 06:24:45 UTC-05:00
4,16462,yashjoshi30052002@gmail.com,Google <no-reply@accounts.google.com>,2step verification turned,image google 2step verification turned yashj...,2025-04-10 11:48:36 UTC
5,16463,yashjoshi30052002@gmail.com,Google <no-reply@accounts.google.com>,security alert,image google app password created sign ...,2025-04-10 11:53:45 UTC
6,16464,yashjoshi30052002@gmail.com,Google <no-reply@accounts.google.com>,security alert,image google app password used sign remo...,2025-04-10 11:59:00 UTC
7,16465,yashjoshi30052002@gmail.com,Google <no-reply@accounts.google.com>,security alert,image google app password created sign ...,2025-04-10 11:59:09 UTC


In [15]:
df1 = df.iloc[:2, :].copy()



for i in range(0, df1.shape[0]):
    print(f'processing row number {i+1}')
    sequence = df1['content'][i]

    type_result = type_classifier(sequence, candidate_labels=type_labels )
    first_key1 = list(type_result.keys())[1]
    first_key2 = list(type_result.keys())[2]
    type_label = type_result[first_key1][0]
    type_value = type_result[first_key2][0]
    df1.loc[i, 'type'] = type_label
    df1.loc[i, 'type_confidence'] = type_value
    print(f'type completed row number {i+1}')

    urgency_result = urgency_classifier(sequence,candidate_labels=urgency_labels)
    first_key1 = list(urgency_result.keys())[1]
    first_key2 = list(urgency_result.keys())[2]
    urgency_label = urgency_result[first_key1][0]
    urgency_value = urgency_result[first_key2][0]
    df1.loc[i, 'urgency'] = urgency_label
    df1.loc[i, 'urgency_confidence'] = urgency_value
    print(f'urgency completed row number {i+1}')

    max_length = 512
    if len(sequence) > max_length:
        sequence = sequence[:max_length]
    sentiment_result = sentiment_classifier(sequence)
    sentiment_label = sentiment_result[0]['label']
    sentiment_value = sentiment_result[0]['score']
    df1.loc[i, 'sentiment'] = sentiment_label
    df1.loc[i, 'sentiment_confidence'] = sentiment_value
    print(f'sentiment completed row number {i+1}')



df1.head(30)

processing row number 1
type completed row number 1
urgency completed row number 1
sentiment completed row number 1
processing row number 2
type completed row number 2
urgency completed row number 2
sentiment completed row number 2


,uid,to,from,subject,content,timestamp,type,confidence,type_confidence,urgency,urgency_confidence,sentiment,sentiment_confidence
0,16458,yashjoshi30052002@gmail.com,Google <no-reply@accounts.google.com>,security alert,image google fetchemail granted access g...,2025-04-10 10:21:16 UTC,technology,None,0.855329,major,0.516962,neutral,0.800549
1,16459,yashjoshi30052002@gmail.com,redBus <greetings@travel.e-redbus.in>,yash plan festive journey :blossom:,redbus welcome plan bus journey new year...,2025-04-10 16:24:19 UTC+05:30,technology,None,0.340465,major,0.613233,neutral,0.634413


In [16]:
df1.to_pickle("type.pkl")